In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 실습 4. Stackoverflow 서베이 기반 인기 언어 찾기

- stackoverflow CSV파일에서 다음 두 필드는 ;를 구분자로 프로그래밍 언어를 구분
    - LanguageHaveWorkedWith
    - LanguageWantToWorkWith
-  이를 별개 레코드로 분리하여 가장 많이 사용되는 언어 top 50와 가장 많이 쓰고 싶은 언어 top 50를 계산해보기
![Stackoverflow.png](Stackoverflow.png)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/usr/local/lib/python3.7/dist-packages/pyspark/jars/RedshiftJDBC42-no-awssdk-1.2.20.1043.jar") \
    .getOrCreate()

In [3]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/survey_results_public.csv

--2023-07-04 16:23:22--  https://s3-geospatial.s3-us-west-2.amazonaws.com/survey_results_public.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.196.249, 3.5.81.145, 52.92.136.90, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.196.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81101949 (77M) [text/csv]
Saving to: ‘survey_results_public.csv’

survey_results_publ 100%[===================>]  77.34M  6.93MB/s    in 12s     

2023-07-04 16:23:35 (6.23 MB/s) - ‘survey_results_public.csv’ saved [81101949/81101949]



In [4]:
!ls -tl

total 82776
-rw-r--r-- 1 jovyan users    24336 Jul  4 16:23 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 jovyan users    67702 Jul  4 16:21 Stackoverflow.png
-rw-r--r-- 1 jovyan users    15543 Jul  4 16:12 PySpark_DataFrame_3.ipynb
drwxr-xr-x 6 jovyan users      192 Jul  4 16:11 extracted.json
drwxr-xr-x 6 jovyan users      192 Jul  4 16:08 extracted_transfer_cost.csv
drwxr-xr-x 5 jovyan users      160 Jul  4 16:06 extracted.csv
-rw-r--r-- 1 jovyan users    24246 Jul  4 15:23 PySpark_DataFrame_2.ipynb
-rw-r--r-- 1 jovyan users    30849 Jul  4 14:58 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 jovyan users    22881 Jul  4 14:30 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 jovyan users    22056 Jul  4 12:19 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 jovyan users    64553 Jul  4 12:19 1800.csv
-rw-r--r-- 1 jovyan users 81101949 Jan 15 03:34 survey_results_public.csv
-rw-r--r-- 1 jovyan users   286779 Apr 24  2022 transfer_cost.txt
-rw-r--r-- 1 jovyan users   146855 Apr 10  2022 customer-orders.csv


In [6]:
!head -5 survey_results_public.csv

ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
1,I am a developer by profession,"Independent contractor, freelancer, or self-employed",Slovakia,NA,NA,"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",18 - 24 years,"Coding Bootcamp;Other online resources (ex: videos, blogs, etc)",NA,NA,"Developer, mobile",20 to 99 employees,EUR Eu

In [7]:
df = spark.read.csv("survey_results_public.csv", header=True).select('ResponseId', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith')

In [8]:
df.printSchema()

root
 |-- ResponseId: string (nullable = true)
 |-- LanguageHaveWorkedWith: string (nullable = true)
 |-- LanguageWantToWorkWith: string (nullable = true)



In [9]:
import pyspark.sql.functions as F

# LanguageHaveWorkedWith 값을 트림하고 ;를 가지고 나눠서 리스트의 형태로 language_have 필드로 설정
df2 = df.withColumn(
    "language_have",
    F.split(F.trim(F.col("LanguageHaveWorkedWith")), ";")
)
# df라는 DataFrame에 language_have라는 새로운 열을 추가합니다. 이 열은 LanguageHaveWorkedWith 열의 값을 세미콜론으로 구분하여 분리한 결과입니다. 
# F.split() 함수는 문자열을 지정된 구분자로 분리하는 데 사용됩니다. 이 함수의 첫 번째 인자는 분리할 문자열, 두 번째 인자는 구분자입니다.
# F.trim() 함수는 문자열의 왼쪽과 오른쪽에서 공백을 제거하는 데 사용됩니다.
# 이 코드의 결과는 df DataFrame에 language_have라는 새로운 열이 추가된 것입니다.

In [11]:
df2.show(5)

+----------+----------------------+----------------------+--------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|       language_have|
+----------+----------------------+----------------------+--------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|[C++, HTML/CSS, J...|
|         2|     JavaScript;Python|                    NA|[JavaScript, Python]|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|[Assembly, C, Pyt...|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|[JavaScript, Type...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|[Bash/Shell, HTML...|
+----------+----------------------+----------------------+--------------------+
only showing top 5 rows



In [14]:
# LanguageWantToWorkWith 값을 트림하고 ;를 가지고 나눠서 리스트의 형태로 language_want 필드로 설정
df3 = df2.withColumn(
    "language_want",
    F.split(F.trim(F.col("LanguageWantToWorkWith")), ";")
)

In [15]:
df3.printSchema()

root
 |-- ResponseId: string (nullable = true)
 |-- LanguageHaveWorkedWith: string (nullable = true)
 |-- LanguageWantToWorkWith: string (nullable = true)
 |-- language_have: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- language_want: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [16]:
df3.show(5)

+----------+----------------------+----------------------+--------------------+--------------------+
|ResponseId|LanguageHaveWorkedWith|LanguageWantToWorkWith|       language_have|       language_want|
+----------+----------------------+----------------------+--------------------+--------------------+
|         1|  C++;HTML/CSS;Java...|                 Swift|[C++, HTML/CSS, J...|             [Swift]|
|         2|     JavaScript;Python|                    NA|[JavaScript, Python]|                [NA]|
|         3|  Assembly;C;Python...|     Julia;Python;Rust|[Assembly, C, Pyt...|[Julia, Python, R...|
|         4|  JavaScript;TypeSc...|  JavaScript;TypeSc...|[JavaScript, Type...|[JavaScript, Type...|
|         5|  Bash/Shell;HTML/C...|  Bash/Shell;HTML/C...|[Bash/Shell, HTML...|[Bash/Shell, HTML...|
+----------+----------------------+----------------------+--------------------+--------------------+
only showing top 5 rows



## 현재 많이 사용되는 언어들 찾기

In [19]:
df_language_have = df3.select(
    df3.ResponseId,
    F.explode(df3.language_have).alias("language_have")
)

'''
이 코드는 df3라는 DataFrame에서 language_have 열의 각 값을 새 행으로 분리합니다. 
한 레코드 내 리스트 개별 언어 레코드를 레코드화한다

예를 들어, language_have 열의 값이 ['Python', 'Java', 'Scala']이면
df_language_have DataFrame의 값은 [('Python', 1), ('Java', 1), ('Scala', 1)]이 됩니다.

이 코드는 Spark SQL의 explode() 함수를 사용합니다. explode() 함수는 배열 또는 컬렉션의 각 값을 새 행으로 분리하는 데 사용됩니다. 
이 함수의 첫 번째 인자는 분리할 배열 또는 컬렉션입니다.

이 코드의 결과는 df3 DataFrame에서 language_have 열의 각 값을 새 행으로 분리한 df_language_have DataFrame입니다.
'''

"\n이 코드는 df3라는 DataFrame에서 language_have 열의 각 값을 새 행으로 분리합니다. \n\n예를 들어, language_have 열의 값이 ['Python', 'Java', 'Scala']이면\ndf_language_have DataFrame의 값은 [('Python', 1), ('Java', 1), ('Scala', 1)]이 됩니다.\n\n이 코드는 Spark SQL의 explode() 함수를 사용합니다. explode() 함수는 배열 또는 컬렉션의 각 값을 새 행으로 분리하는 데 사용됩니다. \n이 함수의 첫 번째 인자는 분리할 배열 또는 컬렉션입니다.\n\n이 코드의 결과는 df3 DataFrame에서 language_have 열의 각 값을 새 행으로 분리한 df_language_have DataFrame입니다.\n"

In [20]:
df_language_have.show(10)

+----------+-------------+
|ResponseId|language_have|
+----------+-------------+
|         1|          C++|
|         1|     HTML/CSS|
|         1|   JavaScript|
|         1|  Objective-C|
|         1|          PHP|
|         1|        Swift|
|         2|   JavaScript|
|         2|       Python|
|         3|     Assembly|
|         3|            C|
+----------+-------------+
only showing top 10 rows



In [21]:
df_language_have.groupby("language_have").count().show(10)

+-------------+-----+
|language_have|count|
+-------------+-----+
|           C#|22984|
|          VBA| 3847|
|         Rust| 5799|
|   Bash/Shell|22385|
|   JavaScript|53587|
|           NA| 1082|
|         Perl| 2028|
|       Erlang|  651|
|       Matlab| 3846|
|      Crystal|  466|
+-------------+-----+
only showing top 10 rows



Sorting 두 가지 방법:

*   sort & orderBy
*   ascending & descending

In [22]:
# sort method를 사용하여 groupby count를 desc 정렬
df_language_have.groupby("language_have").count().sort(F.desc("count")).collect()

[Row(language_have='JavaScript', count=53587),
 Row(language_have='HTML/CSS', count=46259),
 Row(language_have='Python', count=39792),
 Row(language_have='SQL', count=38835),
 Row(language_have='Java', count=29162),
 Row(language_have='Node.js', count=27975),
 Row(language_have='TypeScript', count=24909),
 Row(language_have='C#', count=22984),
 Row(language_have='Bash/Shell', count=22385),
 Row(language_have='C++', count=20057),
 Row(language_have='PHP', count=18130),
 Row(language_have='C', count=17329),
 Row(language_have='PowerShell', count=8871),
 Row(language_have='Go', count=7879),
 Row(language_have='Kotlin', count=6866),
 Row(language_have='Rust', count=5799),
 Row(language_have='Ruby', count=5569),
 Row(language_have='Dart', count=4965),
 Row(language_have='Assembly', count=4632),
 Row(language_have='Swift', count=4204),
 Row(language_have='R', count=4185),
 Row(language_have='VBA', count=3847),
 Row(language_have='Matlab', count=3846),
 Row(language_have='Groovy', count=2479)

In [23]:
# order by 함수를 통해 desc 정렬
df_language_have.groupby("language_have").count().orderBy('count', ascending=False).collect()

[Row(language_have='JavaScript', count=53587),
 Row(language_have='HTML/CSS', count=46259),
 Row(language_have='Python', count=39792),
 Row(language_have='SQL', count=38835),
 Row(language_have='Java', count=29162),
 Row(language_have='Node.js', count=27975),
 Row(language_have='TypeScript', count=24909),
 Row(language_have='C#', count=22984),
 Row(language_have='Bash/Shell', count=22385),
 Row(language_have='C++', count=20057),
 Row(language_have='PHP', count=18130),
 Row(language_have='C', count=17329),
 Row(language_have='PowerShell', count=8871),
 Row(language_have='Go', count=7879),
 Row(language_have='Kotlin', count=6866),
 Row(language_have='Rust', count=5799),
 Row(language_have='Ruby', count=5569),
 Row(language_have='Dart', count=4965),
 Row(language_have='Assembly', count=4632),
 Row(language_have='Swift', count=4204),
 Row(language_have='R', count=4185),
 Row(language_have='VBA', count=3847),
 Row(language_have='Matlab', count=3846),
 Row(language_have='Groovy', count=2479)

In [24]:
df_language50_have = df_language_have.groupby("language_have")\
    .count()\
    .orderBy('count', ascending=False)\
    .limit(50)

In [25]:
df_language50_have.write.mode('overwrite').csv("language50_have")

In [26]:
!ls -tl

total 82784
drwxr-xr-x 6 jovyan users      192 Jul  4 17:00 language50_have
-rw-r--r-- 1 jovyan users    32057 Jul  4 16:58 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 jovyan users    67702 Jul  4 16:21 Stackoverflow.png
-rw-r--r-- 1 jovyan users    15543 Jul  4 16:12 PySpark_DataFrame_3.ipynb
drwxr-xr-x 6 jovyan users      192 Jul  4 16:11 extracted.json
drwxr-xr-x 6 jovyan users      192 Jul  4 16:08 extracted_transfer_cost.csv
drwxr-xr-x 5 jovyan users      160 Jul  4 16:06 extracted.csv
-rw-r--r-- 1 jovyan users    24246 Jul  4 15:23 PySpark_DataFrame_2.ipynb
-rw-r--r-- 1 jovyan users    30849 Jul  4 14:58 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 jovyan users    22881 Jul  4 14:30 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 jovyan users    22056 Jul  4 12:19 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 jovyan users    64553 Jul  4 12:19 1800.csv
-rw-r--r-- 1 jovyan users 81101949 Jan 15 03:34 survey_results_public.csv
-rw-r--r-- 1 jovyan users   286779 Apr 24  2022 transfer_cost.txt
-rw-r--r-- 1 jovyan

In [27]:
!ls -tl language50_have/

total 4
-rw-r--r-- 1 jovyan users   0 Jul  4 17:00 _SUCCESS
-rw-r--r-- 1 jovyan users 447 Jul  4 17:00 part-00000-c7b7acec-6288-4ecb-8b0a-733b69e09573-c000.csv


In [28]:
!cat language50_have/part-00000-c7b7acec-6288-4ecb-8b0a-733b69e09573-c000.csv


JavaScript,53587
HTML/CSS,46259
Python,39792
SQL,38835
Java,29162
Node.js,27975
TypeScript,24909
C#,22984
Bash/Shell,22385
C++,20057
PHP,18130
C,17329
PowerShell,8871
Go,7879
Kotlin,6866
Rust,5799
Ruby,5569
Dart,4965
Assembly,4632
Swift,4204
R,4185
VBA,3847
Matlab,3846
Groovy,2479
Objective-C,2310
Scala,2148
Perl,2028
Haskell,1749
Delphi,1731
Clojure,1552
Elixir,1438
LISP,1096
NA,1082
Julia,1068
F#,804
Erlang,651
APL,536
Crystal,466
COBOL,437


## 가장 배우고 싶은 언어들 찾기

In [29]:
df_language_want = df3.select(
    df3.ResponseId,
    F.explode(df3.language_want).alias("language_want")
)

In [30]:
df_language_want.show(5)

+----------+-------------+
|ResponseId|language_want|
+----------+-------------+
|         1|        Swift|
|         2|           NA|
|         3|        Julia|
|         3|       Python|
|         3|         Rust|
+----------+-------------+
only showing top 5 rows



In [31]:
df_language_want.groupby("language_want").count().show(10)

+-------------+-----+
|language_want|count|
+-------------+-----+
|           C#|17999|
|          VBA| 1069|
|         Rust|15865|
|   Bash/Shell|14043|
|   JavaScript|37008|
|           NA| 6618|
|         Perl| 1175|
|       Erlang| 1379|
|       Matlab| 1562|
|      Crystal|  790|
+-------------+-----+
only showing top 10 rows



In [32]:
df_language50_want = df_language_want.groupby("language_want").count().orderBy('count', ascending=False).limit(50)

In [33]:
df_language50_want.show(10)

+-------------+-----+
|language_want|count|
+-------------+-----+
|   JavaScript|37008|
|       Python|34929|
|     HTML/CSS|29353|
|   TypeScript|26905|
|          SQL|26631|
|      Node.js|24100|
|           C#|17999|
|         Java|17222|
|         Rust|15865|
|           Go|15788|
+-------------+-----+
only showing top 10 rows



In [34]:
df_language50_want.write.mode('overwrite').csv("language50_want")

In [35]:
!ls -tl language50_want/

total 4
-rw-r--r-- 1 jovyan users   0 Jul  4 17:01 _SUCCESS
-rw-r--r-- 1 jovyan users 449 Jul  4 17:01 part-00000-ea60c129-f68e-4b8b-b001-7e915244091a-c000.csv
